In [1]:
import sys
import requests
import pandas as pd
import json
import pyspark.sql.functions as F #import explode_outer, col
from pyspark.sql.types import *
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark import SparkConf
from pyspark import SparkContext
import urllib
from sqlalchemy import __version__ as sa_version, create_engine, text


In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
arquivo = '../DesafioMaguila/DisputasFlamengo.csv'

In [7]:

schema = StructType() \
      .add("Ano",IntegerType(),True) \
      .add("NomCampeonato",StringType(),True) \
      .add("DscSituacao",StringType(),True)


df_desafio = spark.read.options(inferSchema='False'
                                    , delimiter=';'
                                    , encoding="windows-1252") \
                                    .schema(schema) \
                                    .csv(arquivo)

df_desafio.printSchema()

df_desafio.show(50,truncate=False)

root
 |-- Ano: integer (nullable = true)
 |-- NomCampeonato: string (nullable = true)
 |-- DscSituacao: string (nullable = true)

+----+---------------------+-----------+
|Ano |NomCampeonato        |DscSituacao|
+----+---------------------+-----------+
|2019|Campeonato Carioca   |Ganhou     |
|2019|Copa do Brasil       |Perdeu     |
|2019|Copa Libertadores    |Ganhou     |
|2019|Campeonato Brasileiro|Ganhou     |
|2019|Florida CupÂ         |Ganhou     |
|2019|Mundial de Clubes    |Perdeu     |
|2019|TaÃ§a Guanabara      |Perdeu     |
|2019|TaÃ§a Rio            |Ganhou     |
|2020|Campeonato Carioca   |Ganhou     |
|2020|Copa do Brasil       |Perdeu     |
|2020|Copa Libertadores    |Perdeu     |
|2020|Campeonato Brasileiro|Ganhou     |
|2020|Recopa Sul-Americana |Ganhou     |
|2020|Supercopa do Brasil  |Ganhou     |
|2020|TaÃ§a Guanabara      |Ganhou     |
|2020|TaÃ§a Rio            |Perdeu     |
|2021|Campeonato Carioca   |Ganhou     |
|2021|Copa do Brasil       |Perdeu     |
|2021|Cop

In [8]:
df = df_desafio.groupby('Ano') \
                    .pivot("DscSituacao") \
                    .agg(count('DscSituacao') \
                    .alias('Total')) \
                    .sort('Ano') \
                    .fillna(0) 


df.show()

df.select('Ano','Ganhou','Perdeu',expr("""CASE WHEN Ganhou = Perdeu THEN 'Número de Vitórias e Derrotas IGUAL'
                                       WHEN Ganhou > Perdeu THEN 'Ano Vencedor!'
                                       WHEN Perdeu > Ganhou THEN 'Um Ano não tão Vencedor!'
                                       END""").alias('Observacao')) \
                                    .show(truncate=False)

+----+------+------+
| Ano|Ganhou|Perdeu|
+----+------+------+
|2019|     5|     3|
|2020|     5|     3|
|2021|     3|     4|
|2022|     2|     2|
|2023|     0|     2|
+----+------+------+

+----+------+------+-----------------------------------+
|Ano |Ganhou|Perdeu|Observacao                         |
+----+------+------+-----------------------------------+
|2019|5     |3     |Ano Vencedor!                      |
|2020|5     |3     |Ano Vencedor!                      |
|2021|3     |4     |Um Ano não tão Vencedor!           |
|2022|2     |2     |Número de Vitórias e Derrotas IGUAL|
|2023|0     |2     |Um Ano não tão Vencedor!           |
+----+------+------+-----------------------------------+



In [10]:
df = df_desafio.groupBy('DscSituacao') \
                    .agg(count('DscSituacao') \
                    .alias('Total')) \
                    .show()

+-----------+-----+
|DscSituacao|Total|
+-----------+-----+
|     Perdeu|   14|
|     Ganhou|   15|
+-----------+-----+

